In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
cat = ['EDUCATION', 'BUSINESS', 
       'DATING', 'SPORTS', 
       'WEATHER', 'FOOD_AND_DRINK', 
       'HEALTH_AND_FITNESS', 'BEAUTY', 
       'MUSIC_AND_AUDIO', 'NEWS_AND_MAGAZINES', ]

In [ ]:
sports = requests.get('https://play.google.com/store/apps/category/BUSINESS/collection/topselling_free')
sports_html = sports.text #data
sports_data = BeautifulSoup(sports_html)
def get_links(data):
    link_list = []
    for link in data.findAll('a'):
        get_link = link.get('href')
        link_list.append(get_link)
    return link_list
links = get_links(sports_data)
del links[0:107]
sports_links = list(set(links))
sports_links